## Example 1

In [ ]:
library(httr)

url = "https://omabrowser.org/api/protein/P53_RAT/"
response = GET(url)

response_content_list = httr::content(response, as = "parsed")


## Example 2


In [ ]:
library(OmaDB)

sequence='MNDPSLLGYPNVGPQQQQQQQQQQHAGLLGKGTPNALQQQLHMNQLTGIPPPGLMNNSDVHTSSNNNSRQLLDQLANGNANMLNMNMDNNNNNNNNNNNNNNNGGGSGVMMNASTAAVNSIGMVPTVGTPVNINVNASNPLLHPHLDDPSLLNNPIWKLQLHLAAVSAQSLGQPNIYARQNAMKKYLATQQAQQAQQQAQQQAQQQVPGPFGPGPQAAPPALQPTDFQQSHIAEASKSLVDCTKQALMEMADTLTDSKTAKKQQPTGDSTPSGTATNSAVSTPLTPKIELFANGKDEANQALLQHKKLSQYSIDEDDDIENRMVMPKDSKYDDQLWHALDLSNLQIFNISANIFKYDFLTRLYLNGNSLTELPAEIKNLSNLRVLDLSHNRLTSLPAELGSCFQLKYFYFFDNMVTTLPWEFGNLCNLQFLGVEGNPLEKQFLKILTEKSVTGLIFYLRDNRPEIPLPHERRFIEINTDGEPQREYDSLQQSTEHLATDLAKRTFTVLSYNTLCQHYATPKMYRYTPSWALSWDYRRNKLKEQILSYDSDLLCLQEVESKTFEEYWVPLLDKHGYTGIFHAKARAKTMHSKDSKKVDGCCIFFKRDQFKLITKDAMDFSGAWMKHKKFQRTEDYLNRAMNKDNVALFLKLQHIPSGDTIWAVTTHLHWDPKFNDVKTFQVGVLLDHLETLLKEETSHNFRQDIKKFPVLICGDFNSYINSAVYELINTGRVQIHQEGNGRDFGYMSEKNFSHNLALKSSYNCIGELPFTNFTPSFTDVIDYIWFSTHALRVRGLLGEVDPEYVSKFIGFPNDKFPSDHIPLLARFEFMKTNTGSKKV'

seq_annotation = mapSequence(sequence)

seq_annotation$targets[[1]]$omaid          # 'YEAST00058'
seq_annotation$targets[[1]]$oma_hog_id     # 'HOG:0402391.2b'

hog = getHOG(id = 'HOG:0402391.2b',members = TRUE)
hog_members = hog$members


hog = getHOG(id = 'YEAST00058',members = TRUE)
hog_members = hog$members


ontologies = getInfo(hog_members,type=’ontologies’)


library(topGO)

topGO_object = getTopGO(annotations = ontologies, format = 'geneID2GO', myInterestingGenes = hog_members[1:15,])


## Example 3

In [ ]:
library(OmaDB)

tax = getTaxonomy(root = 'Hominoidea')

library(ggtree)

tree = getTree(tax$newick)

mytree = ggtree(tree) 

labels = c("Pan troglodytes",
              "Pan paniscus",
              "Homo sapiens",
              "Gorilla gorilla gorilla",
              "Pongo abelii",
              "Nomascus leucogenys")

library(rphylopic)

ids = sapply(labels, FUN = function(x)         name_search(x)$canonicalName[1,1])

images = sapply(as.character(ids), FUN = function(x) tryCatch(name_images(x)$same[[1]]$uid, error = function(w) name_images(x)$supertaxa[[1]]$uid))


d <- data.frame(label = labels, images = as.character(images))

library(dplyr)

mytree %<+% d + geom_tiplab(aes(image=images), geom="phylopic", offset = 2.3, color='steelblue') + geom_tiplab(offset = 0.3) + ggplot2::xlim(0, 7)



## Example 4

In [ ]:
library(OmaDB)

human_mouse = getGenomeAlignment(genome_id1 =9606, genome_id2 = 10090, per_page = "all", rel_type="1:1")

human_dog = getGenomeAlignment(genome_id1 =9606, genome_id2 = 9615, per_page = "all", rel_type="1:1")

human_mouse$Species  = 'Mus musculus'
human_dog$Species = 'Canis lupus familiaris'

all_pairs = rbind(human_mouse,human_dog)
all_pairs$Species = as.factor(all_pairs$Species)
library(ggplot2)

g <- ggplot(all_pairs, aes(x = 'evolutionary distance [PAM]', fill = Species)) + geom_density(alpha = 0.5) + theme(legend.position="bottom",panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))
dev.off()

print(g)

ks.test(human_dog$distance,human_mouse$distance) 
